# Player Projection Workflow

This notebook prepares checklist features, validates the projection pipeline, and exports the upcoming Sorare projection.

In [ ]:
from pathlib import Path
from pprint import pprint
from statistics import mean

from src.projections.player_game import (
    load_game_logs,
    engineer_features,
    prepare_upcoming_context,
    project_player_game,
    backtest_recent_games,
    write_csv,
)

PROJECT_ROOT = Path('..').resolve()
player_id = 'lebron_james'
GAME_LOG_PATH = PROJECT_ROOT / 'data' / 'game_logs' / f'{player_id}.csv'
CONTEXT_PATH = PROJECT_ROOT / 'data' / 'upcoming_game_context.csv'
PROJECTION_OUTPUT = PROJECT_ROOT / 'outputs' / 'sorare_projections.csv'
BACKTEST_OUTPUT = PROJECT_ROOT / 'outputs' / 'backtest_last5.csv'


In [ ]:
raw_logs = load_game_logs(GAME_LOG_PATH, trailing_games=15)
pprint(raw_logs[-5:])


In [ ]:
feature_logs = engineer_features(raw_logs)
feature_view = [{
    'game_date': row['game_date'].date().isoformat(),
    'minutes': row['minutes'],
    'minutes_avg_5': row['minutes_avg_5'],
    'minutes_trend': row['minutes_trend'],
    'usage_rate': row['usage_rate'],
    'usage_avg_5': row['usage_avg_5'],
    'true_shooting_pct': row['true_shooting_pct'],
    'ts_avg_5': row['ts_avg_5'],
    'pace': row['pace'],
    'pace_avg_5': row['pace_avg_5'],
    'opp_def_avg_5': row['opp_def_avg_5'],
    'sorare_score': row['sorare_score'],
    'sorare_mean_10': row['sorare_mean_10'],
    'sorare_std_10': row['sorare_std_10'],
    'flag_high_pace': row['flag_high_pace'],
    'flag_low_minutes': row['flag_low_minutes'],
    'flag_efficiency_spike': row['flag_efficiency_spike'],
} for row in feature_logs[-5:]]
pprint(feature_view)


In [ ]:
upcoming_context = prepare_upcoming_context(CONTEXT_PATH, player_id)
projection_result = project_player_game(feature_logs, upcoming_context)
pprint(projection_result.to_dict())


In [ ]:
backtest_rows = backtest_recent_games(raw_logs, lookback=5)
backtest_metrics = {
    'MAE': mean(abs(row['projected_score'] - row['actual_score']) for row in backtest_rows),
    'RMSE': (mean((row['projected_score'] - row['actual_score']) ** 2 for row in backtest_rows)) ** 0.5,
    'Mean Minutes Error': mean(row['projection_minutes'] - row['actual_minutes'] for row in backtest_rows),
}
pprint(backtest_rows)
pprint(backtest_metrics)


In [ ]:
projection_rows = [projection_result.to_dict()]
write_csv(PROJECTION_OUTPUT, projection_rows, list(projection_rows[0].keys()))
write_csv(BACKTEST_OUTPUT, backtest_rows, list(backtest_rows[0].keys()))
projection_rows
